# भाग 12: एनक्रिप्टेड डेटावर एनक्रिप्टेड एनएन प्रशिक्षित करा (Train an Encrypted NN on Encrypted Data)

या नोटबुकमध्ये, आम्ही मॉडेल आणि डेटा दोन्ही एनक्रिप्ट केलेले असताना neural नेटवर्क प्रशिक्षण (आणि भविष्यवाणी) करण्यासाठी आपण अद्याप शिकलेली सर्व तंत्रे वापरणार आहोत.

विशेषतः आम्ही आमचे सानुकूल ऑटोग्राड(Autograd) इंजिन सादर केले जे एनक्रिप्टेड संगणनावर कार्य करते.

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)

# चरण 1: कामगार आणि बनावट डेटा तयार करा

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [ ]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# चरण 2: मॉडेल आणि डेटा एनक्रिप्ट करा

येथे एनक्रिप्शण दोन चरणात येते. सिक्युर मल्टी-पार्टी कंप्युटेशन केवळ पूर्णांकांवर कार्य करीत असल्याने दशांश बिंदू (जसे की वजन आणि सक्रियता) सह अधिक संख्येवर कार्य करण्यासाठी आम्हाला निश्चित संख्या वापरुन आपल्या सर्व क्रमांक एनकोड करणे आवश्यक आहे, जे आपल्याला दशांश अचूकतेचे अनेक बिट्स देईल. आपण `.fix_precision()` वर कॉल करून हे करतो.

त्यानंतर आपण आपल्याकडे अन्य डेमोसाठी `.share()` कॉल करू शकतो, जे alice आणि bob मध्ये सामायिक करून सर्व मूल्ये एन्क्रिप्ट करेल. लक्षात ठेवा आपण `requires_grad` ला `True` वर देखील सेट केले आहे, जे एनक्रिप्टेड डेटासाठी एक विशेष `auto_grad` पद्धत देखील जोडते. खरंच, सिक्युअर मल्टी-पार्टी कंप्यूटेशन float व्हॅल्यूजवर कार्य करत नाही, म्हणून आपण नेहमीचा PyTorch autograd वापरू शकत नाही. म्हणून, आपल्याला एक विशेष `AutogradTensor` node जोडण्याची आवश्यकता आहे जी backpropagation साठी gradient graph ची गणना करते. यामध्ये `AutogradTensor` समाविष्ट आहे हे पाहण्यासाठी आपण यापैकी कोणताही घटक मुद्रित करू शकता.

In [ ]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [ ]:
print(data)

# चरण 3: ट्रेन(प्रशिक्षण)

आणि आता आपण साध्या टेन्सर लॉजिकचा वापर करुन प्रशिक्षित करू शकतो.

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

तोटा खरंच कमी झाला!

## निश्चित अचूकतेचा प्रभाव
आपणास आश्चर्य वाटेल की सर्वकाही एन्क्रिप्ट केल्याने कमी होणार्‍या नुकसानावर कसा परिणाम होतो. वास्तविक, सैद्धांतिक गणन समान असल्यामुळे, संख्या एनक्रिप्टेड नसलेल्या प्रशिक्षणाजवळ अगदी जवळ आहे. आपण एनक्रिप्शनशिवाय समान उदाहरण चालवून आणि मॉडेलच्या निर्धारात्मक आरंभसह मॉडेलच्या `__init__` मध्ये हे सत्यापित करू शकता:
```
with torch.no_grad():
    self.fc1.weight.set_(torch.tensor([[ 0.0738, -0.2109],[-0.1579,  0.3174]], requires_grad=True))
    self.fc1.bias.set_(torch.tensor([0.,0.1], requires_grad=True))
    self.fc2.weight.set_(torch.tensor([[-0.5368,  0.7050]], requires_grad=True))
    self.fc2.bias.set_(torch.tensor([-0.0343], requires_grad=True))
```

आपण पहात असलेला थोडा फरक निश्चित परिशुद्धतेमध्ये बदलताना केलेल्या अंकांच्या rounding off मुळे होतो. डीफॉल्ट `precision_fraction` 3 असते आणि आपण ते खाली 2 वर पोहोचल्यास स्पष्ट मजकूर प्रशिक्षणातील विचलन वाढते, तर आपण `precision_fraction = 4` निवडल्यास हे कमी होते.

# अभिनंदन !!! - समुदायात सामील होण्याची वेळ आली!


हे नोटबुक ट्यूटोरियल पूर्ण केल्याबद्दल अभिनंदन! आपण याचा आनंद घेत असल्यास आणि एआय(AI) आणि एआय सप्लाय चेन  (डेटा) च्या विकेंद्रित(Decentralized) मालकीच्या गोपनीयतेच्या संरक्षणाच्या दिशेने चळवळीत सामील होऊ इच्छित असाल तर आपण हे खालील प्रकारे करू शकता!

### Pysyft ला Github वर Star करा!

आमच्या समुदायाला मदत करण्याचा सर्वात सोपा मार्ग म्हणजे फक्त गिटहब(GitHub) रेपो(Repo) तारांकित(Star) करणे! हे आम्ही तयार करीत असलेल्या छान साधनांविषयी जागरूकता वाढविण्यास मदत करते.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### आमच्या Slack मध्ये सामील व्हा!


नवीनतम प्रगतीवर अद्ययावत राहण्याचा उत्तम मार्ग म्हणजे आमच्या समुदायामध्ये सामील होणे! आपण [http://slack.openmined.org](http://slack.openmined.org) येथे फॉर्म भरुन तसे करू शकता.

### एका कोड प्रोजेक्टमध्ये सामील व्हा!

आमच्या समुदायामध्ये योगदानाचा उत्तम मार्ग म्हणजे कोड योगदानकर्ता बनणे! कोणत्याही वेळी आपण (PySyft GitHub Issues Page) वर  जाऊ शकता आणि "Project" साठी फिल्टर करू शकता. हे आपण कोणत्या प्रकल्पांमध्ये सामील होऊ शकता याबद्दल विहंगावलोकन देणारी सर्व उच्च स्तरीय तिकिटे दर्शवेल! आपण एखाद्या प्रकल्पात सामील होऊ इच्छित नसल्यास, परंतु आपण थोडं कोडिंग करू इच्छित असाल तर आपण `good first issues` म्हणून चिन्हांकित गिटहब(GitHub) अंक शोधून आणखी "one off" मिनी-प्रकल्प(mini project) शोधू शकता.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करा

आपल्याकडे आमच्या कोडेबेसमध्ये योगदान देण्यास वेळ नसल्यास, परंतु तरीही आपल्याला समर्थन द्यावयाचे असल्यास आपण आमच्या मुक्त संग्रहात बॅकर देखील होऊ शकता. सर्व देणगी आमच्या वेब होस्टिंग आणि हॅकॅथॉन आणि मेटअप्स सारख्या इतर सामुदायिक खर्चाकडे जातात!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)